#Modules

In [ ]:
import os
import torch 
import numpy as np
import pandas as pd
import h5py
import torch.nn as nn
import scipy
import math 
from scipy import stats
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from google.colab import drive
drive.mount('/content/drive')
LABLE_FILE = '/content/drive/Shareddrives/Summer Intership 2022 - Brain Dataset/data/nsddata/ppdata/subj01/behav/responses.tsv'
FMRI_DIR = '/content/drive/Shareddrives/Summer Intership 2022 - Brain Dataset/data/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR'
FMRI_DIR_STND = '/content/drive/Shareddrives/Summer Intership 2022 - Brain Dataset/data/standardized_betas/subj01'
TRIAL_PER_SESS = 750
# !pip install line_profiler
# %load_ext line_profiler

Mounted at /content/drive


#Debugger

In [ ]:
!pip install ipdb
import ipdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 793 kB 5.2 MB/s 
     |████████████████████████████████| 381 kB 52.6 MB/s 
  Created wheel for ipdb: filename=ipdb-0.13.9-py3-none-any.whl size=11648 sha256=10c865738daeca3b8fcd6f278795ca66d5063fda667bfbbda826b32fbcfdee58
  Stored in directory: /root/.cache/pip/wheels/65/cd/cc/aaf92acae337a28fdd2aa4d632196a59745c8c39f76eaeed01
Successfully built ipdb
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency c

In [ ]:
path = '/content/drive/Shareddrives/Summer Intership 2022 - Brain Dataset/data/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session01.hdf5'

with h5py.File(path, 'r') as f:
  print(type(f['betas'][0][0][0][0]))

<class 'numpy.int16'>


In [ ]:
print('filtering labels...')
res = pd.read_csv(LABLE_FILE, sep='\t')
res = pd.DataFrame(res)
res = res.loc[res['SESSION'] <= 37]
res = res.loc[res['ISOLD'] == 1]
res = res.loc[res['ISCORRECT'] > -1]
# res = res.filter(items = ['ISCORRECT'] )
indx = res.index
res

filtering labels...


,SUBJECT,SESSION,RUN,TRIAL,73KID,10KID,TIME,ISOLD,ISCORRECT,RT,CHANGEMIND,MEMORYRECENT,MEMORYFIRST,ISOLDCURRENT,ISCORRECTCURRENT,TOTAL1,TOTAL2,BUTTON,MISSINGDATA
45,1,1,1,46,28279,4586,0.507350,1,1.0,1104.699282,0.0,28.0,NaN,1,1.0,0,1,2.0,0
68,1,1,2,6,3586,4135,0.510578,1,1.0,782.411240,0.0,3.0,NaN,1,1.0,0,1,2.0,0
90,1,1,2,28,32626,436,0.511689,1,1.0,849.658853,0.0,45.0,NaN,1,1.0,0,1,2.0,0
107,1,1,2,45,33484,5169,0.512569,1,1.0,903.947890,0.0,41.0,NaN,1,1.0,0,1,2.0,0
121,1,1,2,59,15762,2757,0.513263,1,1.0,934.878815,0.0,15.0,NaN,1,1.0,0,1,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27745,1,37,12,58,16201,1257,246.729582,1,1.0,575.265219,0.0,37.0,167.0,1,1.0,0,1,2.0,0
27746,1,37,12,59,55611,7495,246.729629,1,1.0,1664.749701,0.0,3119.0,5135.0,0,0.0,0,1,2.0,0
27747,1,37,12,60,69912,1190,246.729675,1,0.0,1775.520753,0.0,23762.0,NaN,0,1.0,1,0,1.0,0
27748,1,37,12,61,67262,5181,246.729721,1,1.0,517.624946,0.0,135.0,3469.0,1,1.0,0,1,2.0,0


#Standardize all betas in the directory

---



In [ ]:
class StandardizeDirectory:

  def __init__(self, loading_dir, dumping_dir, response_file):
    self.ld = loading_dir
    self.dd = dumping_dir
    self.rs = response_file
    self.offset = 0 #keeps an offset of an index to store in teh hdf5 file
    self.indx = 0
    self.fmri_files = self.getDirFiles()
    self.file_handlers ={}
    self.open_files()

  def __del__(self):
    for key, value in self.file_handlers.items():
      value.close()
  # returns a list of all files in the directory to be standardized
  def getDirFiles(self):
     files = [f for f in os.listdir(self.ld) if 
              os.path.isfile(os.path.join(self.ld, f)) and
              f[-5:] == '.hdf5']
     files.sort()                          
     return files               

  def open_files(self): 
    for file_name in self.fmri_files:
      path = os.path.join(FMRI_DIR, file_name)
      self.file_handlers[file_name] = h5py.File(path, 'r')

  def z_score_voxels(self, sesh, mean, std):
    z_scores = (sesh - mean )/ std
    return z_scores


  # this method removes the unwanted scans from the array passed in
  # Input: an np array of the standadized values of the betas 
  # Output: a dictionary containing (betas, labels, and the offset from zero of the first index)
  def store_data(self,sesh,sesh_num):
    mean = sesh.mean(0)
    std = sesh.std(0)
    std = np.where(std == 0, 0.1, std )
    labels = self.filter_labels(sesh_num) 
    indx = labels.index
    group_dict = {'betas':[],'labels':[],'strt_indx':[] }
    group_dict['strt_indx'].append(self.offset)
    for x in range(len(indx)): 
      group_dict['betas'].append(self.z_score_voxels(sesh[indx[x]-self.indx], mean, std))
      group_dict['labels'].append(labels.at[indx[x], 'ISCORRECT'])
    self.offset = self.offset + len(labels)
    self.indx = self.indx + TRIAL_PER_SESS
    return group_dict
    # Input: path of the responses.tsv file
  # Output: dictionary of indexes as keys with a corresponding class lable
  # in put the sesh number to filter , pass the session as an argument 
  def filter_labels(self, sesh_num):
      print('filtering labels...')
      res = pd.read_csv(self.rs, sep='\t')
      res = pd.DataFrame(res)
      res = res.loc[res['SESSION'] == sesh_num]
      res = res.loc[res['ISOLD'] == 1]
      res = res.loc[res['ISCORRECT'] > -1]
      res = res.filter(items = ['ISCORRECT'] )
      return res

  def dump(self):
    x=0
    for f in self.fmri_files:
      sesh_num = f[-7:-5]
      print('begining dump')
      sesh = np.array(self.file_handlers[f]['betas'])
      ready_data = self.store_data(sesh, int(sesh_num))
      # Open a file
      dump_dir= f'{FMRI_DIR_STND}/stdz_{f}' 
      path = os.path.join(self.dd, dump_dir)
      new_betas = h5py.File(path, 'w')
      print(f'...creating file {x+1}....')
      g1 = new_betas.create_group('betas')
      g2 = new_betas.create_group('labels')
      g3 = new_betas.create_group('startIndx')
      temp = np.array(ready_data['betas'])
      g1.create_dataset('b', data = temp)
      temp = np.array(ready_data['labels'])
      g2.create_dataset('l', data = temp)
      temp = np.array(ready_data['strt_indx'])
      g3.create_dataset('i', data = temp)
      new_betas.close()     
      print(f'DUMPED file {x+1}')
      x = x + 1
      if x > 25:
        break


In [ ]:
st = StandardizeDirectory(FMRI_DIR, FMRI_DIR_STND, LABLE_FILE)
st.dump()

begining dump
filtering labels...
...creating file 1....
DUMPED file 1
begining dump
filtering labels...
...creating file 2....
DUMPED file 2
begining dump
filtering labels...
...creating file 3....
DUMPED file 3
begining dump
filtering labels...
...creating file 4....
DUMPED file 4
begining dump
filtering labels...
...creating file 5....
DUMPED file 5
begining dump
filtering labels...
...creating file 6....
DUMPED file 6
begining dump
filtering labels...
...creating file 7....
DUMPED file 7
begining dump
filtering labels...
...creating file 8....
DUMPED file 8
begining dump
filtering labels...
...creating file 9....
DUMPED file 9
begining dump
filtering labels...
...creating file 10....
DUMPED file 10
begining dump
filtering labels...
...creating file 11....
DUMPED file 11
begining dump
filtering labels...
...creating file 12....
DUMPED file 12
begining dump
filtering labels...
...creating file 13....
DUMPED file 13
begining dump
filtering labels...
...creating file 14....
DUMPED file